In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
file_path = 'Car_Prediction_Splitdata/big/scaled_big_car_used_data_without_outliers.csv'
file_name_with_extension = os.path.basename(file_path)
file_name, file_extension = os.path.splitext(file_name_with_extension)

In [3]:
import csv
import math

data = [
    ['RandomForestRegressor', None, None, None, None, None, None],
    ['DecisionTreeRegressor', None, None, None, None, None, None],
    ['XGBRegressor', None, None, None, None, None, None],
    ['GradientBoostingRegressor', None, None, None, None, None, None],
    ['BaggingRegressor', None, None, None, None, None, None]
]

# Tên tiêu đề cột
header = ['Regressor', 'R2_Score','Cross validation score', 'mean_squared_error', 'mean_absolute_error', 'root_mean_squared_error', 'Diff']

file_name = file_name + '_result.csv'

# Tạo và ghi vào file CSV
with open(file_name, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(data)

In [17]:
def write_result(name,r2_score_value,scores_value,mean_squared_error,mean_absolute_error,root_mean_squared_error,diff_value):
    data = []
    with open(file_name, 'r') as file:
        reader = csv.reader(file)
        data = list(reader)
    for row in data:
        if row[0] == name:
            row[1] = r2_score_value
            row[2] = scores_value
            row[3] = mean_squared_error
            row[4] = mean_absolute_error
            row[5] = root_mean_squared_error
            row[6] = diff_value

    # Ghi lại dữ liệu đã cập nhật vào file CSV
    with open(file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)

In [5]:
df= pd.read_csv('Car_Prediction_Splitdata/big/scaled_big_car_used_data_without_outliers.csv')
df.head()

,maker_model,variants,body_type,type,seat,door,mileage,non_smoker_vehicle,power,gearbox,engine_size,fuel_type,emission_classe,colour,manufacturer_colour,upholstery_colour,upholstery,car_age,inspection_interval,price (€)
0,-1.500732,0.218697,0.432838,0.419615,0.155585,-0.434644,0.001551,-1.731057,1.184217,-0.622976,1.659834,-0.893533,0.676952,-0.013877,-0.329873,1.475672,-0.112568,-0.768208,0.465802,74960
1,1.378892,1.072503,1.892137,0.419615,0.155585,0.747059,1.233923,-1.731057,0.001310,-0.622976,0.167338,-1.030581,-0.730413,0.214151,-0.336679,-0.341382,-0.818723,0.362626,0.465802,22970
2,1.139887,0.625644,1.892137,0.419615,-2.087196,-0.434644,0.080259,0.577682,-0.843624,-0.622976,0.167338,-1.030581,-0.730413,1.126263,-0.916565,-0.341382,1.299740,-0.391263,-0.963469,27999
3,-1.250162,1.602914,0.676054,0.419615,0.155585,-0.434644,0.452509,0.577682,1.860164,-0.622976,1.659834,-0.071245,-0.730413,-1.154017,1.122565,-1.249909,-0.112568,0.362626,0.465802,38499
4,-0.005024,1.528313,0.676054,0.419615,0.155585,0.747059,-0.319185,-1.731057,1.606684,-0.622976,0.211348,-0.208293,0.676952,0.898235,0.299018,-0.341382,1.299740,-0.391263,0.465802,38560


In [6]:
X = df.drop(['price (€)'], axis = 1)
Y = df['price (€)']

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import  GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor

In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error,  r2_score
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42, test_size=.3)
print('Training feature matrix size:',X_train.shape)
print('Training target vector size:',Y_train.shape)
print('Test feature matrix size:',X_test.shape)
print('Test target vector size:',Y_test.shape)

Training feature matrix size: (7315, 19)
Training target vector size: (7315,)
Test feature matrix size: (3136, 19)
Test target vector size: (3136,)


In [10]:
maxAccu=0
maxRS=0
for i in range(1,10):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=i, test_size=.3)
    mod = RandomForestRegressor()
    mod.fit(X_train, Y_train)
    pred = mod.predict(X_test)
    acc=r2_score(Y_test, pred)
    if acc>maxAccu:
        maxAccu=acc
        maxRS=i
print("Best accuracy is ",maxAccu," on Random_state ",maxRS)

Best accuracy is  0.8320922012273809  on Random_state  3


In [18]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
RFR=RandomForestRegressor()
RFR.fit(X_train,Y_train)
pred=RFR.predict(X_test)
R2_score = r2_score(Y_test,pred)*100
print('R2_score:',R2_score)
print('mean_squared_error:',mean_squared_error(Y_test,pred))
print('mean_absolute_error:',mean_absolute_error(Y_test,pred))
print('root_mean_squared_error:',np.sqrt(mean_squared_error(Y_test,pred)))

# Cross Validation Score
scores = cross_val_score(RFR, X, Y, cv = 10).mean()*100
print("\nCross validation score :", scores)

# Difference between Accuracy and CV Score
diff = R2_score - scores
print("\nR2_Score - Cross Validation Score :", diff)

write_result('RandomForestRegressor',R2_score,scores,mean_squared_error(Y_test,pred),mean_absolute_error(Y_test,pred),np.sqrt(mean_squared_error(Y_test,pred)),diff)

R2_score: 82.66941484438189
mean_squared_error: 257438484.01092896
mean_absolute_error: 5439.047658846574
root_mean_squared_error: 16044.889654058981

Cross validation score : 81.82401911156249

R2_Score - Cross Validation Score : 0.845395732819398


In [19]:
XGB=XGBRegressor()
XGB.fit(X_train,Y_train)
pred=XGB.predict(X_test)
R2_score = r2_score(Y_test,pred)*100
print('R2_score:',R2_score)
print('mean_squared_error:',mean_squared_error(Y_test,pred))
print('mean_absolute_error:',mean_absolute_error(Y_test,pred))
print('root_mean_squared_error:',np.sqrt(mean_squared_error(Y_test,pred)))

# Cross Validation on XGB Model
scores = cross_val_score(XGB, X, Y, cv = 10).mean()*100
print("\nCross validation score :", scores)

# Difference between Accuracy and CV Score
diff = R2_score - scores
print("\nR2_Score - Cross Validation Score :", diff)

write_result('XGBRegressor',R2_score,scores,mean_squared_error(Y_test,pred),mean_absolute_error(Y_test,pred),np.sqrt(mean_squared_error(Y_test,pred)),diff)

R2_score: 88.98050293188234
mean_squared_error: 163689950.1260901
mean_absolute_error: 4877.996127887648
root_mean_squared_error: 12794.13733418905

Cross validation score : 85.39067255902012

R2_Score - Cross Validation Score : 3.589830372862224


In [20]:
GBR=GradientBoostingRegressor()
GBR.fit(X_train,Y_train)
pred=GBR.predict(X_test)
R2_score = r2_score(Y_test,pred)*100
print('R2_score:',R2_score)
print('mean_squared_error:',mean_squared_error(Y_test,pred))
print('mean_absolute_error:',mean_absolute_error(Y_test,pred))
print('root_mean_squared_error:',np.sqrt(mean_squared_error(Y_test,pred)))

# Cross Validation on Gradient Boosting
scores = cross_val_score(GBR, X, Y, cv = 10).mean()*100
print("\nCross validation score :", scores)

# Difference between Accuracy and CV Score
diff = R2_score - scores
print("\nR2_Score - Cross Validation Score :", diff)

write_result('GradientBoostingRegressor',R2_score,scores,mean_squared_error(Y_test,pred),mean_absolute_error(Y_test,pred),np.sqrt(mean_squared_error(Y_test,pred)),diff)

R2_score: 76.03586894781915
mean_squared_error: 355976991.7354895
mean_absolute_error: 7019.151876985552
root_mean_squared_error: 18867.352536471288

Cross validation score : 75.99928065514453

R2_Score - Cross Validation Score : 0.03658829267462238


In [21]:
DTR=DecisionTreeRegressor()
DTR.fit(X_train,Y_train)
pred=DTR.predict(X_test)
R2_score = r2_score(Y_test,pred)*100
print('R2_score:',R2_score)
print('mean_squared_error:',mean_squared_error(Y_test,pred))
print('mean_absolute_error:',mean_absolute_error(Y_test,pred))
print('root_mean_squared_error:',np.sqrt(mean_squared_error(Y_test,pred)))
# Cross Validation Score
scores = cross_val_score(DTR, X, Y, cv = 10).mean()*100
print("\nCross validation score :", scores)

# Difference between Accuracy and CV Score
diff = R2_score - scores
print("\nR2_Score - Cross Validation Score :", diff)

write_result('DecisionTreeRegressor',R2_score,scores,mean_squared_error(Y_test,pred),mean_absolute_error(Y_test,pred),np.sqrt(mean_squared_error(Y_test,pred)),diff)

R2_score: 76.09211061084423
mean_squared_error: 355141545.7111678
mean_absolute_error: 7074.297512755102
root_mean_squared_error: 18845.199540232195

Cross validation score : 64.40540772805483

R2_Score - Cross Validation Score : 11.686702882789405


In [22]:
BR=BaggingRegressor()
BR.fit(X_train,Y_train)
pred=BR.predict(X_test)
R2_score = r2_score(Y_test,pred)*100
print('R2_score:',R2_score)
print('mean_squared_error:',mean_squared_error(Y_test,pred))
print('mean_absolute_error:',mean_absolute_error(Y_test,pred))
print('root_mean_squared_error:',np.sqrt(mean_squared_error(Y_test,pred)))

# Cross Validation Score
scores = cross_val_score(BR, X, Y, cv = 10).mean()*100
print("\nCross validation score :", scores)

# Difference between Accuracy and CV Score
diff = R2_score - scores
print("\nR2_Score - Cross Validation Score :", diff)

write_result('BaggingRegressor',R2_score,scores,mean_squared_error(Y_test,pred),mean_absolute_error(Y_test,pred),np.sqrt(mean_squared_error(Y_test,pred)),diff)

R2_score: 78.02791000885651
mean_squared_error: 326386067.58399105
mean_absolute_error: 5861.581058673469
root_mean_squared_error: 18066.15807480913

Cross validation score : 79.78357257971882

R2_Score - Cross Validation Score : -1.755662570862313


In [16]:
from sklearn.model_selection import GridSearchCV